In [1]:
import torch.optim as optim
from torchvision import datasets, transforms
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
from tqdm import tqdm

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
transforms = transforms.Compose([
#     transforms.Resize((300,200)).
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # convert 0-255 to 0-1 and from np to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [4]:
train_path = '/kaggle/input/lung-disease-classification/train_/train_'
test_path = '/kaggle/input/lung-disease-classification/test_/test_'

In [5]:
train_loader = DataLoader(torchvision.datasets.ImageFolder(train_path, transform=transforms), batch_size=16, shuffle=True)
test_loader = DataLoader(torchvision.datasets.ImageFolder(test_path, transform=transforms), batch_size=16, shuffle=True)

In [6]:
root=pathlib.Path(train_path)
# iterate directory of root
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [7]:
print(root)
print(classes)

/kaggle/input/lung-disease-classification/train_/train_
['Bacteria Nenomia', 'COVID-19', 'NORMAL', 'Tuberculosis', 'Virus Nenomia']


In [8]:
class Disease_Classifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size=3)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = nn.Dropout(0.25)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.dropout2 = nn.Dropout(0.25)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.dropout3 = nn.Dropout(0.25)
        
        self.conv4 = nn.Conv2d(128, 64, kernel_size=3)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        self.dropout4 = nn.Dropout(0.25)
        
        self.conv5 = nn.Conv2d(64, 32, kernel_size=3)
        self.relu5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=2)
        self.dropout5 = nn.Dropout(0.25)
        
        
#         self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(896, 128)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(128, 5)  # 5 classes
    
    def forward(self, x):
#         print(x.shape)      #torch.Size([16, 3, 200, 300])
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.dropout1(x)
#         print(x.shape)       #torch.Size([16, 32, 99, 149])
#         print('=================')
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.dropout2(x)        
#         print(x.shape)         #torch.Size([16, 64, 48, 73])
#         print('====================')
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = self.dropout3(x)
#         print(x.shape)          #torch.Size([16, 128, 23, 35])
#         print('==========')
        
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.pool4(x)
        x = self.dropout4(x)
#         print(x.shape)          # torch.Size([16, 64, 10, 16])
#         print('==========')
        
        x = self.conv5(x)
        x = self.relu5(x)
        x = self.pool5(x)
        x = self.dropout5(x)
#         print(x.shape)          #torch.Size([16, 32, 4, 7])
#         print('==========')
        
        x = nn.Flatten()(x)
#         print(x.shape)          #torch.Size([16, 896])
        
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.fc2(x)
        return x

In [9]:
model = Disease_Classifier(num_classes = len(classes)).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [10]:
# Training

In [11]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [12]:
print(train_count,test_count)

2126 33


In [13]:
num_epochs = 10

In [14]:
# Training the model
for epoch in range(num_epochs):
    model.train()
    total_train, correct_train = 0, 0
    for inputs, labels in tqdm(train_loader, desc=f'Training Epoch {epoch}/{num_epochs}', unit='batch'):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted_train = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

    train_accuracy = correct_train / total_train

    # Testing the model
    model.eval()
    correct_test, total_test = 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc=f'Testing {epoch}/{num_epochs}', unit='batch'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted_test = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted_test == labels).sum().item()

    test_accuracy = correct_test / total_test

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {loss.item():.4f}, '
          f'Train Accuracy: {train_accuracy * 100:.2f}%, '
          f'Test Accuracy: {test_accuracy * 100:.2f}%')

Testing 0/10: 100%|██████████| 87/87 [00:09<00:00,  9.39batch/s]


Epoch [1/10], Train Loss: 0.7116, Train Accuracy: 67.33%, Test Accuracy: 66.96%


Testing 1/10: 100%|██████████| 87/87 [00:04<00:00, 18.40batch/s]


Epoch [2/10], Train Loss: 0.2193, Train Accuracy: 75.10%, Test Accuracy: 71.51%


Testing 2/10: 100%|██████████| 87/87 [00:04<00:00, 18.81batch/s]


...

Epoch [8/10], Train Loss: 0.3694, Train Accuracy: 82.70%, Test Accuracy: 78.52%


Testing 8/10: 100%|██████████| 87/87 [00:04<00:00, 18.39batch/s]


Epoch [9/10], Train Loss: 0.8968, Train Accuracy: 82.95%, Test Accuracy: 79.61%


Testing 9/10: 100%|██████████| 87/87 [00:04<00:00, 18.34batch/s]

Epoch [10/10], Train Loss: 0.1702, Train Accuracy: 83.72%, Test Accuracy: 79.25%
